<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os
import re

In [2]:
path = r'Data/100-0.txt'

In [3]:
data = []
with open(path, 'r', encoding='utf8') as f:
    data.append(f.read())

In [4]:
data = [re.sub('\ufeff', ' ', i) for i in data]
data = [re.sub('\n', ' ', i) for i in data]
data = [re.sub(r"\'", "'", i) for i in data]
data[0][:500]

' THE SONNETS                     1  From fairest creatures we desire increase, That thereby beauty’s rose might never die, But as the riper should by time decease, His tender heir might bear his memory: But thou contracted to thine own bright eyes, Feed’st thy light’s flame with self-substantial fuel, Making a famine where abundance lies, Thy self thy foe, to thy sweet self too cruel: Thou that art now the world’s fresh ornament, And only herald to the gaudy spring, Within thine own bud buriest '

In [5]:
giant_string = ' '.join(data)
giant_string[:500]

' THE SONNETS                     1  From fairest creatures we desire increase, That thereby beauty’s rose might never die, But as the riper should by time decease, His tender heir might bear his memory: But thou contracted to thine own bright eyes, Feed’st thy light’s flame with self-substantial fuel, Making a famine where abundance lies, Thy self thy foe, to thy sweet self too cruel: Thou that art now the world’s fresh ornament, And only herald to the gaudy spring, Within thine own bud buriest '

In [6]:
chars = list(set(giant_string))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [7]:
indices_char = int_char
char_indices = char_int

In [8]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in giant_string]

sequences = []
next_chars = []

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i: i + maxlen])
    next_chars.append(encoded[i + maxlen])

print('sequences: ', len(sequences))

sequences:  1096509


In [9]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1

    y[i, next_chars[i]] = 1

In [10]:
x.shape, y.shape

((1096509, 40, 99), (1096509, 99))

In [11]:
text = giant_string

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(y.shape[1], activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen)
    for diversity in [0.2, 0.5, 0.75, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(500):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [16]:
model.fit(x, y,
          batch_size=512,
          epochs=4,
          callbacks=[print_callback])

Epoch 1/4
1096192/1096509 [============================>.] - ETA: 0s - loss: 1.7003
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "t a verdict?   ALL. No more talking on't"
t a verdict?   ALL. No more talking on't the words the words     That the compand and the words the sweet the with of the word.     The good the wittle shall be such be the words     And the wittle the witch and be the would be my lord.     And a prince the what is my soul and the word of the words     That the would speak of the words and the words     That shall be so such be the word of the word.     The would be should be the would be a prince of the way the weach a boy.     The would be such my lord and the witting the way the wo
----- diversity: 0.5
----- Generating with seed: "t a verdict?   ALL. No more talking on't"
t a verdict?   ALL. No more talking on't and fair bear the word.                                                                                          

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


 brook me for him, But in her almost for her did, and host to so be I began, Lood you was sent; and the fury the part for the fee.     Let it on me she res 'twist him of thy press.     In man save me for the soldiers and time To wounding good weard to pearly of thy rances.     Marry, and no was wear his frow.   CLEOPH. To go of the applamon of shadow and you have him? O Farewell, have progressies of his where’st no best decain,     In thy hearty parts of men in the ground.   BENT. Then 
----- diversity: 1.0
----- Generating with seed: "e, And tell his wife that, being lunatic"
e, And tell his wife that, being lunatic wretch a’en, When she did well sawy to laugh and boy'd and weep. If you shall ge:     And never hold hy after for lands, wrong     All pand, a Adices is the wonding of city,     Like this daughter of this else; Te! I was her is laption we defecuid to him,     keep !   The bagions kill shalt the judging to this?   TITUS. [Myselp, the PISCOOO? ARVITbian that Gentlefock.]  ’R

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN